In [86]:
import pandas as pd
import numpy as np

In [87]:
df = pd.read_csv("./Batch_5064543_batch_results_50.csv")

In [88]:
neded_cols = ['HITId', 'HITTypeId' 'Input.utterance']
for i in range(0,9):
    feature = 'Answer.object' + str(i)
    neded_cols.append(feature)
for i in range(0,9):
    feature = 'Answer.order' + str(i)
    neded_cols.append(feature)

df_filtered = df.groupby('HITId')
HITIds = [item for item, index in df_filtered.groups.items()]

def check_submatches(lst, orders):
    max_count = 0
    longest_subset = []
    for i in range(len(lst)):
        count = 1
        subset = []
        for j in range(i+1, len(lst)):
            if len(lst[i]) == len(lst[j]):
                submatch = True
                for k in range(len(lst[i])):
                    if lst[i][k] not in lst[j][k] and lst[j][k] not in lst[i][k]:
                        # print('wrong submatch:', lst[i][k], lst[j][k])
                        submatch = False
                        break
                    try:
                        if submatch and int(orders[i][k]) != int(orders[j][k]):
                          submatch = False
                    except:
                        submatch = False
                    # if submatch and orders[i][k] != orders[j][k]:
                    #     submatch = False
                        # print(f'{lst[i]} and {lst[j]} have a submatch but orders are wrong: {orders[i]}, {orders[j]}.')

                if submatch:
                    count += 1
                    subset.append(j)
                    # print(f'{lst[i]} and {lst[j]} have a submatch.')
        # print('old', max_count, 'new', count)
        if max_count < count:
          max_count = count
          longest_subset = subset
        # max_count = max(max_count, count)
        # if 
    return max_count, longest_subset

matches_3 = 0 
matches_4 = 0
total_3 = 0
total_4 = 0
clean_hits = []
clean_hits3 = []
match3_index = {}
# loop over the groups
for hit_id, indices in df_filtered.groups.items():
    # extract the rows for the current group
    group_df = df.loc[indices, :]
    total_orders = []
    total_objects = []
    if len(indices) == 3:
        total_3 += 1
    elif len(indices) == 4:
        total_4 += 1
        # print(hit_id)

    for i in range(len(indices)):
      orders = []
      objects = []
      for j in range(1,9):
        feature1 = 'Answer.order' + str(j)
        feature2 = 'Answer.object' + str(j)
        if str(group_df[feature1].iloc[i]) != 'nan':
            orders.append(group_df[feature1].iloc[i])
        
        if str(group_df[feature2].iloc[i]) != 'nan':
            object_ = str(group_df[feature2].iloc[i])
            object_ = object_.lower()
            object_ = object_.translate(str.maketrans('', '', string.punctuation))
            objects.append(object_)
      total_orders.append(orders)
      total_objects.append(objects)

    num_matches, subset = check_submatches(total_objects, total_orders)
    # if hit_id == '3ZXV7Q5FJB7GMRDOQMQTNZ7MQMRCFA':
    #   print("how do i have", num_matches)
    #   print(total_objects)
    #   print(total_orders)
    #   break
    if num_matches == 4:
        clean_hits.append(hit_id)
        matches_4 +=1
    elif num_matches == 3 and len(indices) == 4:
        clean_hits3.append(hit_id)
        match3_index[hit_id] = subset
        matches_3 += 1
    # print(total_objects)
    # print(total_orders)
    # print('-'*10)

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.id',
       'Input.stimulus_id', 'Input.utterance', 'Input.correct_guess',
       'Input.speaker_id', 'Input.listener_id', 'Input.scan_id',
       'Input.instance_type', 'Input.target_id', 'Input.tokens',
       'Input.dataset', 'Input.mentions_target_class',
       'Input.uses_object_lang', 'Input.uses_spatial_lang',
       'Input.uses_color_lang', 'Input.uses_shape_lang', 'Answer.object1',
       'Answer.object2', 'Answer.object3', 'Answer.o

'\nThe needed columns for querying: \n    1. HITTId-> The unique ID for the sentence \n    2. Input.utterance -> Sentence\n    3. Input.Instance  -> Target\n    4. ِ[Answer.object1, Answer.object8] -> Objects\n    5. [Answer.order1, Answer.order8] -> Order\n'